# Build Datasets

I made the corpus here: [kaggle.com/mauroebordon/creating-a-qa-corpus-from-askreddit/](https://www.kaggle.com/mauroebordon/creating-a-qa-corpus-from-askreddit/)


In [7]:
import pandas as pd
import re
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop = stopwords.words('english')

db_df = pd.read_csv("db/ask-reddit-corpus.csv", index_col=0)

## Feature Extraction

Only Tokens, Lemmas, PoS AND stopword filtering for now.


In [8]:

def extract_features(df: pd.DataFrame, toks=True, lems=True, pos=True):

    """
    idea: las collocations
    """
    lmtzr = WordNetLemmatizer()
   
    #Filtramos las preguntas demasiado extensas
    db_df = df.copy()[df.Q.apply(lambda x: len(str(x)) <50)]

    #remove stopwords
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')        
    db_df["Qless"] = db_df.Q.str.replace(pattern, '')
    db_df["Aless"] = db_df.ANS.str.replace(pattern, '')
        
    #Obtenemos los tokens
    if toks:
        db_df["Qtoks"] = [word_tokenize(w) for w in db_df["Qless"]]
        db_df["Atoks"] = [word_tokenize(w) for w in db_df["Aless"]]

    #Obtenemos los lemmas
    if lems:
        db_df["Qlemmas"] = [' '.join(lmtzr.lemmatize(t) for t in qes) for qes in db_df["Qtoks"]]
        db_df["Alemmas"] = [' '.join(lmtzr.lemmatize(t) for t in qes) for qes in db_df["Atoks"]]


    # Par Tok & POS
    if pos:
        db_df["Qpos"] = [pos_tag(word_tokenize(w)) for w in db_df["Qless"]]
        db_df["Apos"] = [pos_tag(word_tokenize(w)) for w in db_df["Aless"]]

    
    
    db_df = db_df[["id", "Q", "Qscore", "Qless", "Qlemmas", "Qpos", "Qtoks", "ANS", "ANSscore", "Aless", "Alemmas", "Apos", "Atoks"]]

    #eliminamos los repetidos
    db_df = db_df.groupby("Q", as_index=False).first()


    #solo consideramos con las preguntas con 2 o más upvotes (~32k)
    #sdf = df[df.Qscore > 1]

    #filtramos las respuestas muy cortas, nos deja un total de 25k. 
    #sdf = sdf.copy()[sdf.ANS.apply(lambda x: len(str(x)) > 15)]


    return db_df
    

#usar pickle no sirve para comprimir naranja
df = extract_features(db_df)
df

In [ ]:
df.to_csv("db/features.csv")

# Building the Test Dataset for Clustering Comparition

Small manual clusters to get a little sence if clustering is working

Esto quizas es muy muy tonto, pero solamente ordenar alfabeticamente las preguntas es una buena forma de encontrar similaridad.


In [ ]:
import pandas as pd

df = pd.read_csv("db/features.csv", index_col=0)

n_clus = 5 #no cambiar sin agregar otro
n_sam = 100

df = df.groupby("Q", as_index=False).first()

#testeo de algunos "tipos" de preguntas precatios y de concepto
q_ids = []
q_ids += list(df[df.Q.str.contains(r'favorite movie')].id.sample(n_sam))
q_ids += list(df[df.Q.str.contains(r'ever seen')].id.sample(n_sam))
q_ids += list(df[df.Q.str.contains(r'advice')].id.sample(n_sam))
q_ids += list(df[df.Q.str.contains(r'history')].id.sample(n_sam))
q_ids += list(df[df.Q.str.contains(r'book')].id.sample(n_sam))

# Mostramos un poquito como queda
# for j in range(n_clus):
#    print(f"\ncluster {j}")
#    for i in range(n_sam):
#       print(f"   {df.loc[q_ids[j*n_clus+i]].Q}")


q_ids = [(x, int(i/n_sam)) for i, x in enumerate(q_ids)] 

total = n_clus*n_sam

test_db = []
for i in range(total):
    for j in range(1, total-i):
        este = q_ids[i]
        otro = q_ids[j+i]
        test_db += [[este[0], otro[0], este[1] == otro[1]]]


test_db = pd.DataFrame(test_db)
test_db = test_db[test_db[0] != test_db[1]]


In [ ]:
test_db

In [ ]:
test_db.to_csv("db/test_db.csv")